In [1]:
!pip install kagglehub

You should consider upgrading via the '/Users/mmckay/Desktop/Skill_Development/chest_xray_CNN_training/tf-gpu-env/bin/python3 -m pip install --upgrade pip' command.


In [2]:
# import kagglehub

In [3]:
# # Download latest version
# path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")
# work_dir = "/Users/mmckay/Desktop/Skill_Development/chest_xray_CNN_training/DATA"

# print(work_dir, path)

In [4]:
# from keras.models import Sequential
# from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
# from keras.preprocessing.image import ImageDataGenerator
# from keras.optimizers import SGD

# def Build_CNN_Model():
#     model = Sequential()
#     # Add a 2D Convolutional layer with 32 filters and a 3x3 kernel
#     model.add(Conv2D(32, (3, 3), padding='same', input_shape=(224, 224, 1)))
#     # Remove all values that are negative and change to zero
#     model.add(Activation('relu'))
#     # Add a 2D MaxPooling layer with a 2x2 pool size(reduced function and take the maximum values in the pool)
#     model.add(MaxPooling2D(pool_size=(2, 2)))

#     # -----------------------------------------------------
#     # Add a 2D Convolutional layer with 64 filters and a 3x3 kernel
#     model.add(Conv2D(64, (3, 3), padding='same'))
#     # Remove all values that are negative and change to zero
#     model.add(Activation('relu'))
#     # Add a 2D MaxPooling layer with a 2x2 pool size(reduced function and take the maximum values in the pool)
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     # Add a 2D Convolutional layer with 128 filters and a 3x3 kernel

#     # -----------------------------------------------------
#     model.add(Conv2D(128, (3, 3), padding='same'))
#     # Remove all values that are negative and change to zero
#     model.add(Activation('relu'))
#     # Add a 2D MaxPooling layer with a 2x2 pool size(reduced function and take the maximum values in the pool)
#     model.add(MaxPooling2D(pool_size=(2, 2)))

#     # -----------------------------------------------------
#     # Apply flattening to the model - reduce from high deminsionallty to 1D(flat column)
#     model.add(Flatten())
#     # Add a dense layer with 1000 neurons
#     model.add(Dense(1000))
#     # Add a relu activation function to the dense layer
#     model.add(Activation('relu'))
#     # Add softmax activation function to the output layer for classification
#     model.add(Dense(2))
#     model.add(Activation('softmax'))
#     # Display the model summary
#     model.summary()

#     # Compile the model with categorical crossentropy loss function and SGD optimizer
#     opt = SGD(learning_rate=0.001)
#     model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
#     return model

# Build_CNN_Model()

In [17]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Activation, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.optimizers import SGD


def Build_CNN_Model():
    model = Sequential()
    # Add a 2D Convolutional layer with 32 filters and a 3x3 kernel
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=(224, 224, 1)))
    # Remove all values that are negative and change to zero
    model.add(Activation('relu'))
    # Add a 2D MaxPooling layer with a 2x2 pool size (reduced function and take the maximum values in the pool)
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # -----------------------------------------------------
    # Add a 2D Convolutional layer with 64 filters and a 3x3 kernel
    model.add(Conv2D(64, (3, 3), padding='same'))
    # Remove all values that are negative and change to zero
    model.add(Activation('relu'))
    # Add a 2D MaxPooling layer with a 2x2 pool size (reduced function and take the maximum values in the pool)
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # -----------------------------------------------------
    # Add a 2D Convolutional layer with 128 filters and a 3x3 kernel
    model.add(Conv2D(128, (3, 3), padding='same'))
    # Remove all values that are negative and change to zero
    model.add(Activation('relu'))
    # Add a 2D MaxPooling layer with a 2x2 pool size (reduced function and take the maximum values in the pool)
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Apply flattening to the model - reduce from high deminsionallty to 1D(flat column)
    model.add(Flatten())
    # Add a dense layer with 1000 neurons
    model.add(Dense(1000))
    # Add a relu activation function to the dense layer
    model.add(Activation('relu'))
    # Add softmax activation function to the output layer for classification
    model.add(Dense(2))
    model.add(Activation('softmax'))

    # Display the model summary
    model.summary()

    # Compile the model with categorical crossentropy loss function and SGD optimizer
    opt = SGD(learning_rate=0.001)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# Check GPU availability
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Example of using GPU
if len(tf.config.experimental.list_physical_devices('GPU')) > 0:
    with tf.device('/GPU:0'):
        model = Build_CNN_Model()
else:
    print("No GPU available, building model on CPU")
    model = Build_CNN_Model()

Num GPUs Available:  1
Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 224, 224, 32)      320       
                                                                 
 activation_26 (Activation)  (None, 224, 224, 32)      0         
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 112, 112, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_25 (Conv2D)          (None, 112, 112, 64)      18496     
                                                                 
 activation_27 (Activation)  (None, 112, 112, 64)      0         
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 56, 56, 64)       0         
 g2D)                          

In [18]:
def Train_CNN_Model(model):
    # Model Parameters
    batch_size = 32
    epochs = 100
    # Data Augmentation fro grayscale xray images
    # Data normalize by scaleing and shifting and normalizing the data
    # - Scaling: making the pixel repersented as a float between 0 and 255 to a float between 0 and 1
    # - Zero Mean Shifting: shifting the data to have a mean of 0
    # - Normalization: scaling the data to have a standard deviation of 1 using the unit variance
    # from keras.preprocessing.image import ImageDataGenerator
    train_datagen =  ImageDataGenerator(rescale=1./255,featurewise_center=True, featurewise_std_normalization=True)
    valid_datagen =  ImageDataGenerator(rescale=1./255,featurewise_center=True, featurewise_std_normalization=True)

    # Load the training data
    train_generator = train_datagen.flow_from_directory(
            '/Users/mmckay/Desktop/Skill_Development/chest_xray_CNN_training/DATA/train',
            classes=['NORMAL', 'adnormal'],
            target_size=(224, 224), 
            color_mode='grayscale',
            batch_size=batch_size,
            class_mode='categorical')

    # Load the validation data
    valid_generator = valid_datagen.flow_from_directory(
            '/Users/mmckay/Desktop/Skill_Development/chest_xray_CNN_training/DATA/val',
            classes=['NORMAL', 'adnormal'],
            target_size=(224, 224),
            color_mode='grayscale',
            batch_size=batch_size,
            class_mode='categorical')
    
    
    history = model.fit(train_generator, steps_per_epoch=len(train_generator), epochs=epochs, validation_data=valid_generator, validation_steps=len(valid_generator), verbose=1)

    # Accuracy and Loss
    import matplotlib.pyplot as plt
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')  
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

    # Save the model
    model.save('chest_xray_cnn.h5')

In [6]:
# model = Build_CNN_Model()
# Train_CNN_Model(model)

In [7]:
# import tensorflow as tf

# print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


In [19]:
model = Build_CNN_Model()
Train_CNN_Model(model)

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_27 (Conv2D)          (None, 224, 224, 32)      320       
                                                                 
 activation_31 (Activation)  (None, 224, 224, 32)      0         
                                                                 
 max_pooling2d_27 (MaxPoolin  (None, 112, 112, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_28 (Conv2D)          (None, 112, 112, 64)      18496     
                                                                 
 activation_32 (Activation)  (None, 112, 112, 64)      0         
                                                                 
 max_pooling2d_28 (MaxPoolin  (None, 56, 56, 64)       0         
 g2D)                                                 

2024-12-03 15:05:45.483205: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2024-12-03 15:05:46.323277: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-12-03 15:05:46.802493: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignVariableOp.


42/42 [==============================] - ETA: 0s - loss: 0.1482 - accuracy: 0.9761

2024-12-03 15:07:02.204599: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2024-12-03 15:07:02.418326: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


42/42 [==============================] - 78s 2s/step - loss: 0.1482 - accuracy: 0.9761 - val_loss: 0.0362 - val_accuracy: 1.0000
Epoch 2/100
42/42 [==============================] - 32s 765ms/step - loss: 0.0203 - accuracy: 1.0000 - val_loss: 0.0152 - val_accuracy: 1.0000
Epoch 3/100
42/42 [==============================] - 113s 3s/step - loss: 0.0098 - accuracy: 1.0000 - val_loss: 0.0091 - val_accuracy: 1.0000
Epoch 4/100
42/42 [==============================] - 70s 2s/step - loss: 0.0062 - accuracy: 1.0000 - val_loss: 0.0064 - val_accuracy: 1.0000
Epoch 5/100
42/42 [==============================] - 51s 1s/step - loss: 0.0044 - accuracy: 1.0000 - val_loss: 0.0049 - val_accuracy: 1.0000
Epoch 6/100
42/42 [==============================] - 65s 2s/step - loss: 0.0034 - accuracy: 1.0000 - val_loss: 0.0039 - val_accuracy: 1.0000
Epoch 7/100
42/42 [==============================] - 92s 2s/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 0.0033 - val_accuracy: 1.0000
Epoch 8/100
42/42 [==

KeyboardInterrupt: 

In [20]:
# !pip install pillow

You should consider upgrading via the '/Users/mmckay/Desktop/Skill_Development/chest_xray_CNN_training/tf-gpu-env/bin/python3 -m pip install --upgrade pip' command.
